<b>5. 실습을 통한 이해</b><br>
- LDA는 gensim 사용

1) 정수 인코딩과 단어 집합 만들기
- LSA챕터에서 사용한 Twenty Newsgroups 데이터 및 전처리 과정을 활용
- tokenized_doc으로 저장한 상태부터 시작함

In [2]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers','footeers','quotes'))
documents = dataset.data
len(documents)
print(dataset.target_names)

# 텍스트 전처리
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]"," ")
# 길이가 3이하인 단어 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

# 불용어 제거
from nltk.corpus import stopwords
# NLTK로부터 불용어 받아옴
stop_words = stopwords.words('english')
# 토큰화
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
# 불용어 제거
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [3]:
# 훈련용 뉴스 5개만 출력
tokenized_doc[:5]

0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

- 각 단어에 정수 인코딩을 하는 동시에 각 뉴스에서의 단어 빈도수를 기록
- 각 단어를 (word_id, word_frequency)의 형태로 바꿈
- word_id는 단어가 정수 인코딩된 값이고 word_frequency는 해당 뉴스에서의 해당 단어의 빈도수를 의미
- 이는 gensim의 corpora.Dictionary()를 사용

In [4]:
# pip install gensim

from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력, 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 2), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1)]


- 두 번째 뉴스의 출력 결과, (66, 2)는 정수 인코딩이 66으로 할당된 단어가 두번째 뉴스에서 두번 등장

In [5]:
# 66이라는 단어가 어떤 단어인지 확인
print(dictionary[66])

dostoevsky


In [6]:
# 총 학습된 단어의 개수 확인
len(dictionary)

70484

2) LDA 모델 훈련시키기
- 토픽 개수를 20으로 하여 학습

In [7]:
import gensim
NUM_TOPICS = 20 # 토픽 개수 k = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.020*"game" + 0.018*"team" + 0.014*"games" + 0.014*"year"')
(1, '0.025*"space" + 0.011*"nasa" + 0.007*"data" + 0.005*"program"')
(2, '0.008*"colormap" + 0.008*"mask" + 0.007*"germany" + 0.007*"sweden"')
(3, '0.015*"jesus" + 0.009*"christian" + 0.008*"bible" + 0.008*"church"')
(4, '0.028*"university" + 0.012*"research" + 0.012*"health" + 0.010*"medical"')
(5, '0.014*"government" + 0.013*"president" + 0.010*"state" + 0.010*"states"')
(6, '0.009*"would" + 0.007*"much" + 0.007*"like" + 0.007*"good"')
(7, '0.009*"slave" + 0.008*"doug" + 0.008*"master" + 0.007*"dyer"')
(8, '0.013*"fire" + 0.010*"never" + 0.010*"koresh" + 0.009*"tobacco"')
(9, '0.029*"period" + 0.017*"power" + 0.013*"play" + 0.013*"scorer"')
(10, '0.013*"file" + 0.008*"program" + 0.008*"windows" + 0.007*"available"')
(11, '0.010*"food" + 0.006*"cubs" + 0.005*"kevin" + 0.004*"scores"')
(12, '0.026*"georgia" + 0.019*"water" + 0.013*"rutgers" + 0.011*"gatech"')
(13, '0.011*"bike" + 0.008*"engine" + 0.007*"good" + 0.006*"ri

- 각 단어 앞에 붙은 수치 : 단어의 해당 토픽에 대한 기여도
- passes : 알고리즘의 동작 횟수 (알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 적당한 횟수 지정)
- num_words=4 : 총 4개의 단어만 출력

# LDA 시각화 하기
- 시각화 : pip install pyLDAvis

import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [8]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i == 5:
        break
    print(i, '번째 문서의 topic 비율은', topic_list)

0 번째 문서의 topic 비율은 [(2, 0.017979898), (4, 0.09672378), (14, 0.48671848), (15, 0.09900035), (16, 0.11860768), (18, 0.16947204)]
1 번째 문서의 topic 비율은 [(3, 0.08425105), (11, 0.19295196), (12, 0.020964203), (14, 0.29460165), (18, 0.39221802)]
2 번째 문서의 topic 비율은 [(4, 0.051485468), (5, 0.02315645), (6, 0.017746411), (14, 0.4417252), (17, 0.025380848), (18, 0.36311036), (19, 0.06711812)]
3 번째 문서의 topic 비율은 [(1, 0.041216128), (5, 0.06328405), (6, 0.26983684), (7, 0.030355416), (11, 0.015257737), (14, 0.05285601), (17, 0.029109165), (18, 0.1637964), (19, 0.3261525)]
4 번째 문서의 topic 비율은 [(0, 0.4325652), (6, 0.26096943), (18, 0.2781178)]


- (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포
- 예를 들어, 0번째 문서의 토픽에서 (2, 0.017979898)은 2번 토픽이 1.7%의 분포를 (14, 0.48671848)는 14번 토픽이 48.7%의 분포

In [9]:
# 데이터 프레임 형식으로 출력
def make_topictable_per_doc(ldamodel, corpus, texts):
    topic_table = pd.DataFrame()
    
    #몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내옴
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 정렬
        # 예) 정렬 전 0번 문서 : [(2번 토픽, 1.7%), (4번 토픽, 9.6%), (14번 토픽, 48.7%) 였다면]
        #     정렬 후 0번 문서 : [(14번 토픽, 48.7%), (4번 토픽, 9.6%), (2번 토픽, 1.7%) 순으로 정렬]
        
        # 모든 문서에 대해서 각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc):  # 몇 번 토픽인지와 비중을 나눠서 저장
            if j == 0:  # 정렬이 완료된 상태이므로 가장 앞에 있는 것이 비중이 가장 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic, 4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과 비중 그리고 전체 토픽의 비중을 저장
            else:
                break
    return(topic_table)

In [11]:
topictable = make_topictable_per_doc(ldamodel, corpus, tokenized_doc)
print(topictable[:10])
topictable = topictable.reset_index()  # 문서 번호를 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만듬
print("================================================================")
print(topictable[:10])

      0       1                                                  2
0  14.0  0.4867  [(2, 0.01797956), (4, 0.09671218), (14, 0.4867...
1  18.0  0.3923  [(3, 0.08424776), (11, 0.1929529), (12, 0.0209...
2  14.0  0.4417  [(4, 0.051490474), (5, 0.023157403), (6, 0.017...
3  19.0  0.3262  [(1, 0.04121628), (5, 0.0632835), (6, 0.269845...
4   0.0  0.4326  [(0, 0.43258402), (6, 0.26110706), (18, 0.2779...
5   3.0  0.5310  [(2, 0.017816834), (3, 0.53102016), (6, 0.0819...
6  10.0  0.3815  [(6, 0.26749086), (10, 0.38151368), (15, 0.036...
7  18.0  0.4917  [(5, 0.055641364), (8, 0.034521755), (11, 0.01...
8  18.0  0.3335  [(4, 0.051082034), (7, 0.17797029), (10, 0.182...
9  18.0  0.3927  [(2, 0.015176013), (4, 0.14343473), (6, 0.2628...
   index     0       1                                                  2
0      0  14.0  0.4867  [(2, 0.01797956), (4, 0.09671218), (14, 0.4867...
1      1  18.0  0.3923  [(3, 0.08424776), (11, 0.1929529), (12, 0.0209...
2      2  14.0  0.4417  [(4, 0.051490474)

In [14]:
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,14.0,0.4867,"[(2, 0.01797956), (4, 0.09671218), (14, 0.4867..."
1,1,18.0,0.3923,"[(3, 0.08424776), (11, 0.1929529), (12, 0.0209..."
2,2,14.0,0.4417,"[(4, 0.051490474), (5, 0.023157403), (6, 0.017..."
3,3,19.0,0.3262,"[(1, 0.04121628), (5, 0.0632835), (6, 0.269845..."
4,4,0.0,0.4326,"[(0, 0.43258402), (6, 0.26110706), (18, 0.2779..."
5,5,3.0,0.5310,"[(2, 0.017816834), (3, 0.53102016), (6, 0.0819..."
6,6,10.0,0.3815,"[(6, 0.26749086), (10, 0.38151368), (15, 0.036..."
7,7,18.0,0.4917,"[(5, 0.055641364), (8, 0.034521755), (11, 0.01..."
8,8,18.0,0.3335,"[(4, 0.051082034), (7, 0.17797029), (10, 0.182..."
9,9,18.0,0.3927,"[(2, 0.015176013), (4, 0.14343473), (6, 0.2628..."
